In [9]:
import matplotlib
%matplotlib inline
matplotlib.rcParams['font.size'] = 28.0
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2
import loadMWSnap as mws
import velocitySpectroscopyHelpers as vsh
reload(mws);
reload(vsh);

In [18]:
# load and do chi2
haloes = vsh.haloesWithVSData
for i, h in enumerate(haloes):
    print 'Halo %d of %d:' % (i+1, len(haloes))
    nb, an, fill = vsh.loadSpectroscopyData(halo=h)
    
    # the chi-square value for this fit
    # No weights -- already taken care of in Ns**-0.5 in the uncertainties!
#     x2 = np.average((nb['cen']-an['cen'])**2/(nb['sig']**2+an['sig']**2), weights=None)
    x2 = np.sum((nb['cen']-an['cen'])**2/(nb['sig']**2+an['sig']**2))
    dof = len(nb['l'])
    
#     # TODO: treating DOF??

    print 'Chi^2 =', x2, 'p = ', chi2.cdf(x2, dof)

Halo 1 of 18:
Chi^2 = 0.0671895940335 p =  2.64810687545e-11
Halo 2 of 18:
Chi^2 = 0.0238115652685 p =  8.97583092642e-14
Halo 3 of 18:
Chi^2 = 0.00457899691754 p =  1.04354498245e-17
Halo 4 of 18:
Chi^2 = 0.112783732681 p =  4.48497742624e-10
Halo 5 of 18:
Chi^2 = 0.0230886283982 p =  7.57815534269e-14
Halo 6 of 18:
Chi^2 = 0.0212568542941 p =  4.81343083935e-14
Halo 7 of 18:
Chi^2 = 0.0212955904255 p =  4.86179244486e-14
Halo 8 of 18:
Chi^2 = 0.00676521819531 p =  8.92118437998e-17
Halo 9 of 18:
Chi^2 = 0.0259727901866 p =  1.44609326626e-13
Halo 10 of 18:
Chi^2 = 0.0168372104016 p =  1.33815965159e-14
Halo 11 of 18:
Chi^2 = 0.0781432832575 p =  6.04877442716e-11
Halo 12 of 18:
Chi^2 = 0.0103630692345 p =  9.29817588291e-16
Halo 13 of 18:
Chi^2 = 0.0101718849547 p =  8.39372238707e-16
Halo 14 of 18:
Chi^2 = 0.0130464414553 p =  3.29552764466e-15
Halo 15 of 18:
Chi^2 = 0.0336555193567 p =  5.99435423838e-13
Halo 16 of 18:
Chi^2 = 0.0160392179889 p =  1.02488776692e-14
Halo 17 of 18:
C

In [ ]:
# Plot it up!
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
halo = 374
nb, an, fill = vsh.loadSpectroscopyData(halo=halo)

# sample the N-body distribution and get a chi^2 for this sampling
x2 = 0.0
dof = 0
meanphot = nb['Ns'] + nb['Nb']

# np.random.seed(10291989)
# meanphot *= 10

# recover the actual line widths for the analytic signal
# include background for the observed signal
# an['sig'] *= vsh.cramer_rao(sig=an['Ns'], bg=an['Nb'])**-1*an['Ns']**0.5
# nb['sig'] *= nb['Ns']**0.5

nphot = np.random.poisson(lam=meanphot)
for i, l in enumerate(nb['l']):
    en = np.random.normal(loc=nb['cen'][i], scale=nb['sig'][i], size=nphot[i])
    ax.scatter(l*np.ones_like(en), en*10**3, lw=0)
    x2 += np.sum(((en-an['cen'][i])/an['sig'][i])**2)
    dof += nphot[i]
print 'chi2 =', x2, 'dof =', dof
print 'prob =', chi2.cdf(x2, dof)
    
    
    
ax.plot([-120, 120], [0,0], 'k-', lw=0.4)

# inds=[0,2,4,6,8,10]
# ax.errorbar(nb['l'][inds], nb['cen'][inds]*10**3,\
#             yerr=nb['sig'][inds]*10**3, fmt='o', c='blue', label='N-body')


# fill['sig'] *= vsh.cramer_rao(sig=fill['Ns'], bg=fill['Nb'])**-1*fill['Ns']**0.5

flon = fill['l']
cent = fill['cen']*10**3
plus = cent + fill['sig']*10**3
minus = cent - fill['sig']*10**3
ax.fill_between(flon, minus, plus, facecolor='black', alpha=0.08, label='Analytic')
ax.plot(flon, cent, 'k-', alpha=0.1)
# ax.plot([-120, 120], [0,0], 'k-', lw=0.4)


# frills
# ax.set_title(r'Decay line spectral shift')
ax.set_ylabel(r'$\frac{\delta E}{E} \times 10^3$')
ax.set_xlabel(r'$\ell$ ($^{\circ}$)')
ax.set_xlim(-120., 120.)
ax.invert_xaxis()
ax.set_ylim(-1.2, 1.2)
# ax.set_ylim(-1.5, 2)
ax.grid(True)

# ax.legend(loc='upper left')

# fig.savefig('../img/de_vs_l_800.png', bbox_inches='tight')
# fig.savefig('../img/de_vs_l_filled.png', bbox_inches='tight')